In [140]:
#数据处理
import math
import random

import numpy as np
#读取写入数据
import pandas as pd
import os
import csv

from sympy import false
from sympy.stats import Moment
from tornado.escape import squeeze
#进度条
from tqdm import tqdm
#pytorch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset,random_split
# 读取数据
train_df = pd.read_csv('covid.train.csv')
test_df = pd.read_csv('covid.test.csv')
train_df.fillna(train_df.mean(), inplace=True)
print(f"训练集大小: {len(train_df)}, 测试集大小: {len(test_df)}")


训练集大小: 2700, 测试集大小: 893


In [141]:
# 将 DataFrame 转换为 Tensor
class COVIDataset(Dataset):
    def __init__(self, dataframe,targets=None):
        self.target = targets
        if targets:
            self.features = torch.FloatTensor(dataframe.iloc[:, :-1].values)
            self.labels = torch.LongTensor(dataframe.iloc[:, -1].values)
        else:
            self.features = torch.FloatTensor(dataframe.iloc[:, :-1].values)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        if self.target:
            return self.features[idx], self.labels[idx]
        else:
            return self.features[idx]


In [142]:
    # 创建自定义 Dataset
train_dataset = COVIDataset(train_df,1)
test_dataset = COVIDataset(test_df,None)
train_dataset.features.shape[1], train_dataset.labels.shape,test_dataset.features.shape

(94, torch.Size([2700]), torch.Size([893, 93]))

In [143]:
#设置随机种子
def same_seed(seed):
    torch.backends.mkl.deterministic=True
    torch.backends.mkl.benchmark=False
    np.random.seed(seed)
    torch.manual_seed(seed)
same_seed(5201314)

In [144]:
# 分割训练集和验证集
valid_size = int(len(train_dataset) * 0.2)
train_size = len(train_dataset) - valid_size
train_data, valid_data = random_split(
    train_dataset,
    [train_size, valid_size],
    generator=torch.manual_seed(5201314)
)

In [145]:
# 创建 DataLoader
batch_size = 300
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [146]:
# 查看数据
# 神经网络
class My_Model(nn.Module):
    def __init__(self,input_dim):
        super(My_Model, self).__init__()
        self.layers=nn.Sequential(
            nn.Linear(input_dim,16),
            nn.ReLU(),
            nn.Linear(16,8),
            nn.ReLU(),
            nn.Linear(8,1),
        )

    def forward(self, x):
        x = self.layers(x)
        x = squeeze(1)
        return x

In [149]:
device = 'cuda'if torch.cuda.is_available() else 'cpu'
config = {
    'seed':5201314,
    'select_all':True,
    'valid_ratio':0.2,
    'n_epochs':3000,
    'batch_size':256,
    'learning_rate':1e-5,
    'early_stop':400,
    'save_path':'/models/model.ckpt'
}

In [151]:
if not os.path.isdir('/models'):
    os.mkdir('/models')

In [156]:
import os

# 定义要检查的路径
target_path = '/models'

# 检查路径是否存在
if os.path.exists(target_path):
    print(f"路径 '{target_path}' 存在。")
    # 检查路径是否为目录
    if os.path.isdir(target_path):
        print(f"'{target_path}' 是一个目录。")
        print(f"{current_dir}")
    else:
        print(f"'{target_path}' 不是一个目录。")
else:
    print(f"路径 '{target_path}' 不存在。")

路径 '/models' 存在。
'/models' 是一个目录。


NameError: name 'current_dir' is not defined